<a href="https://colab.research.google.com/github/jsuarezzz8/Proyecto2_MOS/blob/main/Proyecto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import sys
import os


if 'google.colab' in sys.modules:

    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'


    !apt-get install -y -qq glpk-utils
    !apt-get install -y coinor-libhighs-dev
    !pip install gurobipy
    !pip install amplpy pyomo -q
    !python -m amplpy.modules install coin highs scip gcg -q

    !pip install pyomo
    !pip install folium
    !pip install polyline
    !pip install branca
    !apt-get install -y coinor-cbc
# Crear archivo de licencia
    os.makedirs("/root/gurobi", exist_ok=True)
    with open("/root/gurobi/gurobi.lic", "w") as f:
        f.write("""LICENSEID=2662152
TYPE=ACADEMIC
VERSION=12
HOSTNAME=DESKTOP-M317T6F
HOSTID=40c938df
USERNAME=usuario
EXPIRATION=2026-05-07
""")
os.environ['GRB_LICENSE_FILE'] = "/root/gurobi/gurobi.lic"
os.environ['PATH'] += ":/opt/gurobi120/gurobi1201/linux64/bin"


In [ ]:
import csv

def leer_datos_centro(archivo_centros):
    centros = {}

    # Leer centros
    with open(archivo_centros, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            centros["CD"+row['DepotID']] = {
                'lat': float(row['Latitude']),
                'lon': float(row['Longitude']),

            }

    return centros

def leer_datos_clientes( archivo_clientes):
    clientes = {}

    # Leer clientes
    with open(archivo_clientes, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            clientes["C"+row['ClientID']] = {
                'lat': float(row['Latitude']),
                'lon': float(row['Longitude']),
                'demanda': float(row['Demand'])
            }

    return clientes

def leer_datos_vehiculos(archivo_vehiculos):
    vehiculos = {}


    # Leer vehículos
    with open(archivo_vehiculos, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            vehiculos["V"+row['VehicleID']] = {
                'capacidad': float(row['Capacity']),
                'rango': float(row['Range'])
            }

    return  vehiculos

def leer_datos_centro_caso1(archivo_centros):
    centros = {}

    with open(archivo_centros, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        tiene_capacidad = 'Capacity' in reader.fieldnames  # Verifica si la columna existe

        for row in reader:
            capacidad = float(row['Capacity']) if tiene_capacidad else float('inf')
            centros["CD"+row['DepotID']] = {
                'lat': float(row['Latitude']),
                'lon': float(row['Longitude']),
                'capacidad': capacidad
            }

    return centros

In [ ]:

# Puedes cambiar estos datos fácilmente
centros = leer_datos_centro_caso1('depots.csv')
clientes = leer_datos_clientes('clients.csv')
vehiculos = leer_datos_vehiculos('vehicles.csv')


print(centros)
print(clientes)
print(vehiculos)


{'CD1': {'lat': 4.743359, 'lon': -74.153536, 'capacidad': inf}}
{'C1': {'lat': 4.59795431125545, 'lon': -74.09893796560621, 'demanda': 13.0}, 'C2': {'lat': 4.687820646838871, 'lon': -74.07557103763986, 'demanda': 15.0}, 'C3': {'lat': 4.70949446000624, 'lon': -74.10708524062704, 'demanda': 12.0}, 'C4': {'lat': 4.605029068682624, 'lon': -74.09727965657427, 'demanda': 15.0}, 'C5': {'lat': 4.648463876533332, 'lon': -74.16464148202755, 'demanda': 20.0}, 'C6': {'lat': 4.662137416953968, 'lon': -74.12083799988112, 'demanda': 17.0}, 'C7': {'lat': 4.697499030379109, 'lon': -74.02213076607309, 'demanda': 17.0}, 'C8': {'lat': 4.649416884236942, 'lon': -74.17207549744595, 'demanda': 20.0}, 'C9': {'lat': 4.606310650273935, 'lon': -74.15615257246444, 'demanda': 20.0}, 'C10': {'lat': 4.557379705282216, 'lon': -74.09041145358674, 'demanda': 15.0}, 'C11': {'lat': 4.591594072172954, 'lon': -74.17802255204528, 'demanda': 17.0}, 'C12': {'lat': 4.7564172406324055, 'lon': -74.1015410917749, 'demanda': 12.0}

In [ ]:
import pyomo.environ as pyo
import time
from math import sqrt

def get_euclidean_distance(lat1, lon1, lat2, lon2):
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2) * 111

def modelo(centros, clientes, vehiculos):
    tiempo_inicio = time.time()
    nodos = list(centros.keys()) + list(clientes.keys())
    coordenadas = {**centros, **clientes}

    distancias = {
        (i, j): get_euclidean_distance(coordenadas[i]['lat'], coordenadas[i]['lon'],
                                       coordenadas[j]['lat'], coordenadas[j]['lon'])
        for i in nodos for j in nodos if i != j
    }

    P_f, F_t, C_m, alpha = 15000, 5000, 700, 0.025
    costo_km = F_t + alpha * P_f + C_m

    model = pyo.ConcreteModel()

    model.N = pyo.Set(initialize=nodos)
    model.C = pyo.Set(initialize=clientes.keys())
    model.D = pyo.Set(initialize=centros.keys())
    model.V = pyo.Set(initialize=vehiculos.keys())

    model.Q = pyo.Param(model.V, initialize=lambda m, v: vehiculos[v]['capacidad'])
    model.R = pyo.Param(model.V, initialize=lambda m, v: vehiculos[v]['rango'])
    model.q = pyo.Param(model.C, initialize=lambda m, i: clientes[i]['demanda'])

    model.Arcos = pyo.Set(initialize=[(i, j, v) for i in nodos for j in nodos if i != j for v in vehiculos])

    model.x = pyo.Var(model.Arcos, domain=pyo.Binary)
    model.y = pyo.Var(model.D, model.V, domain=pyo.Binary)
    model.u = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals, bounds=(1, len(clientes)))

    def objetivo(model):
        return sum(costo_km * distancias[i, j] * model.x[i, j, v] for (i, j, v) in model.Arcos)
    model.obj = pyo.Objective(rule=objetivo, sense=pyo.minimize)

    model.asignacion = pyo.Constraint(model.V, rule=lambda m, v: sum(m.y[k, v] for k in m.D) == 1)

    def visita_una_vez(m, j):
        return sum(m.x[i, j, v] for i in m.N for v in m.V if (i, j, v) in m.Arcos) == 1
    model.visita = pyo.Constraint(model.C, rule=visita_una_vez)

    def inicio(m, k, v):
        return sum(m.x[k, j, v] for j in m.N if (k, j, v) in m.Arcos) == m.y[k, v]
    model.inicio = pyo.Constraint(model.D, model.V, rule=inicio)

    def fin(m, k, v):
        return sum(m.x[j, k, v] for j in m.N if (j, k, v) in m.Arcos) == m.y[k, v]
    model.fin = pyo.Constraint(model.D, model.V, rule=fin)

    def flujo(m, j, v):
        return sum(m.x[i, j, v] for i in m.N if (i, j, v) in m.Arcos) == \
               sum(m.x[j, i, v] for i in m.N if (j, i, v) in m.Arcos)
    model.flujo = pyo.Constraint(model.C, model.V, rule=flujo)

    def capacidad(m, v):
        return sum(m.q[i] * sum(m.x[j, i, v] for j in m.N if (j, i, v) in m.Arcos) for i in m.C) <= m.Q[v]
    model.capacidad = pyo.Constraint(model.V, rule=capacidad)

    def autonomia(m, v):
        return sum(distancias[i, j] * m.x[i, j, v] for (i, j, vv) in m.Arcos if vv == v) <= m.R[v]
    model.autonomia = pyo.Constraint(model.V, rule=autonomia)

    def mtz(m, i, j, v):
        if i != j and i in m.C and j in m.C and (i, j, v) in m.Arcos:
            return m.u[i, v] - m.u[j, v] + len(m.C) * m.x[i, j, v] <= len(m.C) - 1
        return pyo.Constraint.Skip
    model.mtz = pyo.Constraint(model.C, model.C, model.V, rule=mtz)


    solver = pyo.SolverFactory('cbc')
    solver.options['seconds'] = 600

    results = solver.solve(model, tee=True)

    print(f"\n⏱ Tiempo total: {time.time() - tiempo_inicio:.2f} segundos")
    print(f"📉 Mejor valor encontrado: {pyo.value(model.obj):,.2f}")



    return model


In [ ]:
import folium
import random
from folium import plugins

def mostrar_mapa_solucion(model, centros, clientes, vehiculos):
    # Crear mapa centrado en el primer centro
    centro_inicio = next(iter(centros.values()))
    mapa = folium.Map(location=[centro_inicio['lat'], centro_inicio['lon']], zoom_start=13)

    # Colores únicos para cada vehículo
    colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen']
    color_vehiculo = {v: colores[i % len(colores)] for i, v in enumerate(model.V)}

    # Agregar centros
    for k in centros:
        coord = centros[k]
        folium.Marker([coord['lat'], coord['lon']], popup=f"CD: {k}", icon=folium.Icon(color='black', icon='home')).add_to(mapa)

    # Agregar clientes
    for c in clientes:
        coord = clientes[c]
        folium.Marker([coord['lat'], coord['lon']], popup=f"Cliente: {c}\nDemanda: {clientes[c]['demanda']}", icon=folium.Icon(color='gray', icon='user')).add_to(mapa)

    # Dibujar rutas
    for (i, j, v) in model.Arcos:
        if pyo.value(model.x[i, j, v]) == 1:
            coord_i = centros[i] if i in centros else clientes[i]
            coord_j = centros[j] if j in centros else clientes[j]
            folium.PolyLine(
                locations=[(coord_i['lat'], coord_i['lon']), (coord_j['lat'], coord_j['lon'])],
                color=color_vehiculo[v],
                weight=4,
                tooltip=f"{v}: {i} → {j}"
            ).add_to(mapa)

    # Leyenda de colores por vehículo
    legend_html = """
    <div style='position: fixed; bottom: 20px; left: 20px; z-index:9999; background-color:white;
                padding: 10px; border:2px solid grey; border-radius:6px'>
      <h4 style='margin-top: 0'>Vehículos</h4>
    """
    for v in model.V:
        legend_html += f"<i style='background:{color_vehiculo[v]};width:10px;height:10px;display:inline-block;'></i> {v}<br>"
    legend_html += "</div>"
    mapa.get_root().html.add_child(folium.Element(legend_html))

    return mapa




In [ ]:
# Puedes cambiar estos datos fácilmente
centros = leer_datos_centro_caso1('depots.csv')
clientes = leer_datos_clientes('clients.csv')
vehiculos = leer_datos_vehiculos('vehicles.csv')


print(centros)
print(clientes)
print(vehiculos)

model = modelo(centros,clientes,vehiculos)
mapa = mostrar_mapa_solucion(model, centros, clientes, vehiculos)
mapa.save("mapa_rutas.html")
mapa


{'CD1': {'lat': 4.743359, 'lon': -74.153536, 'capacidad': inf}}
{'C1': {'lat': 4.59795431125545, 'lon': -74.09893796560621, 'demanda': 13.0}, 'C2': {'lat': 4.687820646838871, 'lon': -74.07557103763986, 'demanda': 15.0}, 'C3': {'lat': 4.70949446000624, 'lon': -74.10708524062704, 'demanda': 12.0}, 'C4': {'lat': 4.605029068682624, 'lon': -74.09727965657427, 'demanda': 15.0}, 'C5': {'lat': 4.648463876533332, 'lon': -74.16464148202755, 'demanda': 20.0}, 'C6': {'lat': 4.662137416953968, 'lon': -74.12083799988112, 'demanda': 17.0}, 'C7': {'lat': 4.697499030379109, 'lon': -74.02213076607309, 'demanda': 17.0}, 'C8': {'lat': 4.649416884236942, 'lon': -74.17207549744595, 'demanda': 20.0}, 'C9': {'lat': 4.606310650273935, 'lon': -74.15615257246444, 'demanda': 20.0}, 'C10': {'lat': 4.557379705282216, 'lon': -74.09041145358674, 'demanda': 15.0}, 'C11': {'lat': 4.591594072172954, 'lon': -74.17802255204528, 'demanda': 17.0}, 'C12': {'lat': 4.7564172406324055, 'lon': -74.1015410917749, 'demanda': 12.0}


⏱ Tiempo total: 608.12 segundos
📉 Mejor valor encontrado: 1,265,806.29


In [ ]:

# Puedes cambiar estos datos fácilmente
centros = leer_datos_centro_caso1('depotsC2.csv')
clientes = leer_datos_clientes('clientsC2.csv')
vehiculos = leer_datos_vehiculos('vehiclesC2.csv')


print(centros)
print(clientes)
print(vehiculos)

model = modelo(centros,clientes,vehiculos)
mapa = mostrar_mapa_solucion(model, centros, clientes, vehiculos)
mapa.save("mapa_rutas.html")
mapa

{'CD1': {'lat': 4.75021190869025, 'lon': -74.08124218159384, 'capacidad': 8.0}, 'CD2': {'lat': 4.5363832206427785, 'lon': -74.10993358606953, 'capacidad': 10.0}, 'CD3': {'lat': 4.792925960208614, 'lon': -74.03854814565923, 'capacidad': 0.0}, 'CD4': {'lat': 4.72167778077445, 'lon': -74.06706883098641, 'capacidad': 4.0}, 'CD5': {'lat': 4.607707046760958, 'lon': -74.13826337931849, 'capacidad': 28.0}, 'CD6': {'lat': 4.650463053612691, 'lon': -74.12400186370824, 'capacidad': 3.0}, 'CD7': {'lat': 4.621911772492814, 'lon': -74.09561875464892, 'capacidad': 0.0}, 'CD8': {'lat': 4.678960680833056, 'lon': -74.10975623736951, 'capacidad': 10.0}, 'CD9': {'lat': 4.735973062153282, 'lon': -74.09547235719887, 'capacidad': 43.0}, 'CD10': {'lat': 4.550640992537941, 'lon': -74.10991610076434, 'capacidad': 1.0}, 'CD11': {'lat': 4.664702960902753, 'lon': -74.10977422186991, 'capacidad': 16.0}, 'CD12': {'lat': 4.5791740634103135, 'lon': -74.12408925943333, 'capacidad': 18.0}}
{'C1': {'lat': 4.6325528404247

In [ ]:
import pyomo.environ as pyo
import time
from math import sqrt

def calcular_distancias(nodos, coordenadas):
    """Calcula matriz de distancias (km) entre todos los nodos."""
    distancias = {}
    for i in nodos:
        for j in nodos:
            if i != j:
                dx = coordenadas[i]['lat'] - coordenadas[j]['lat']
                dy = coordenadas[i]['lon'] - coordenadas[j]['lon']
                distancias[(i, j)] = sqrt(dx**2 + dy**2) * 111
    return distancias

def verificar_factibilidad(centros, clientes, vehiculos):
    """Verifica si el problema es factible antes de resolver."""
    deposito = next(iter(centros.keys()))

    # Verificar capacidades
    demanda_total = sum(clientes[c]['demanda'] for c in clientes)
    capacidad_total = sum(vehiculos[v]['capacidad'] for v in vehiculos)

    print(f"📊 Verificación de factibilidad:")
    print(f"   Demanda total: {demanda_total}")
    print(f"   Capacidad total: {capacidad_total}")
    print(f"   Ratio: {demanda_total/capacidad_total:.2%}")

    if demanda_total > capacidad_total:
        print("❌ PROBLEMA NO FACTIBLE: Demanda > Capacidad total")
        return False

    # Verificar si algún cliente tiene demanda > capacidad máxima
    max_capacidad = max(vehiculos[v]['capacidad'] for v in vehiculos)
    max_demanda = max(clientes[c]['demanda'] for c in clientes)

    if max_demanda > max_capacidad:
        print(f"❌ PROBLEMA NO FACTIBLE: Cliente con demanda {max_demanda} > capacidad máxima {max_capacidad}")
        return False

    print("✅ Problema parece factible")
    return True

# Función principal que maneja ambos casos
def modelo_super_rapido(centros, clientes, vehiculos):
    """Modelo ultra-simplificado para obtener solución en < 1 minuto."""
    tiempo_inicio = time.time()

    deposito = next(iter(centros.keys()))
    print(f"🔍 MODO SÚPER RÁPIDO: {len(clientes)} clientes, {len(vehiculos)} vehículos")

    # Modelo simplificado con menos variables
    model = pyo.ConcreteModel()
    model.C = pyo.Set(initialize=list(clientes.keys()))
    model.V = pyo.Set(initialize=list(vehiculos.keys()))

    # Variables principales
    model.x = pyo.Var(model.C, model.V, domain=pyo.Binary)  # Cliente i asignado a vehículo v
    model.y = pyo.Var(model.V, domain=pyo.Binary)  # Vehículo v usado

    # Parámetros
    model.Q = pyo.Param(model.V, initialize={v: vehiculos[v]['capacidad'] for v in vehiculos})
    model.q = pyo.Param(model.C, initialize={c: clientes[c]['demanda'] for c in clientes})

    # Calcular distancias depot-cliente
    dist_depot = {}
    for c in clientes:
        dx = centros[deposito]['lat'] - clientes[c]['lat']
        dy = centros[deposito]['lon'] - clientes[c]['lon']
        dist_depot[c] = sqrt(dx**2 + dy**2) * 111

    costo_por_km = 5000 + 0.025 * 15000 + 700

    # Función objetivo simplificada (solo ida y vuelta al depot)
    def objetivo_simple(m):
        return sum(2 * costo_por_km * dist_depot[c] * m.x[c, v]
                  for c in m.C for v in m.V)

    model.obj = pyo.Objective(expr=objetivo_simple(model), sense=pyo.minimize)

    # Restricciones básicas
    model.asignar_cliente = pyo.Constraint(
        model.C,
        rule=lambda m, c: sum(m.x[c, v] for v in m.V) == 1
    )

    model.capacidad = pyo.Constraint(
        model.V,
        rule=lambda m, v: sum(m.q[c] * m.x[c, v] for c in m.C) <= m.Q[v]
    )

    model.activar_vehiculo = pyo.Constraint(
        model.V,
        rule=lambda m, v: sum(m.x[c, v] for c in m.C) <= len(m.C) * m.y[v]
    )

    # Solver muy agresivo
    solver = pyo.SolverFactory('highs')
    solver.options = {
        'time_limit': 60,  # Solo 1 minuto
        'mip_rel_gap': 0.2,  # 20% gap está bien para velocidad
        'presolve': 'aggressive',
        'parallel': 'on',
        'mip_heuristic_effort': 0.8,  # Máximas heurísticas
        'output_flag': 0  # Sin output para ir más rápido
    }

    print("⚡ Resolviendo modelo súper rápido...")

    try:
        results = solver.solve(model, tee=False, load_solutions=False)

        if results.solver.termination_condition in [pyo.TerminationCondition.optimal,
                                                   pyo.TerminationCondition.feasible]:
            model.solutions.load_from(results)
            tiempo_total = time.time() - tiempo_inicio
            print(f"✅ Solución encontrada en: {tiempo_total:.2f} segundos")
            print(f"💰 Costo aproximado: ${pyo.value(model.obj):,.2f}")
            return model

    except Exception as e:
        print(f"❌ Error: {e}")

In [ ]:
import folium
import random
from folium import plugins
import pyomo.environ as pyo

def mostrar_mapa_solucion_simple(model, centros, clientes, vehiculos):
    """
    Función de visualización compatible con el modelo simplificado.
    Solo muestra asignaciones cliente-vehículo y rutas al depósito.
    """
    # Crear mapa centrado en el primer centro (depósito)
    deposito = next(iter(centros.keys()))
    centro_inicio = centros[deposito]
    mapa = folium.Map(location=[centro_inicio['lat'], centro_inicio['lon']], zoom_start=13)

    # Colores únicos para cada vehículo
    colores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen', 'pink', 'gray']
    color_vehiculo = {v: colores[i % len(colores)] for i, v in enumerate(model.V)}

    # Agregar depósito
    folium.Marker(
        [centro_inicio['lat'], centro_inicio['lon']],
        popup=f"Depósito: {deposito}",
        icon=folium.Icon(color='black', icon='home', prefix='fa')
    ).add_to(mapa)

    # Procesar asignaciones y crear rutas
    rutas_por_vehiculo = {}

    for v in model.V:
        rutas_por_vehiculo[v] = []
        carga_total = 0

        # Encontrar clientes asignados a este vehículo
        for c in model.C:
            if pyo.value(model.x[c, v]) == 1:
                rutas_por_vehiculo[v].append(c)
                carga_total += clientes[c]['demanda']

        # Solo procesar vehículos que tienen clientes asignados
        if rutas_por_vehiculo[v]:
            print(f"🚛 Vehículo {v}: {len(rutas_por_vehiculo[v])} clientes, carga: {carga_total}/{vehiculos[v]['capacidad']}")

            # Agregar marcadores para clientes de este vehículo
            for c in rutas_por_vehiculo[v]:
                coord = clientes[c]
                folium.Marker(
                    [coord['lat'], coord['lon']],
                    popup=f"Cliente: {c}<br>Demanda: {clientes[c]['demanda']}<br>Vehículo: {v}",
                    icon=folium.Icon(color=color_vehiculo[v], icon='user', prefix='fa')
                ).add_to(mapa)

                # Dibujar línea del depósito al cliente
                folium.PolyLine(
                    locations=[
                        (centro_inicio['lat'], centro_inicio['lon']),
                        (coord['lat'], coord['lon'])
                    ],
                    color=color_vehiculo[v],
                    weight=3,
                    opacity=0.7,
                    tooltip=f"Vehículo {v} → Cliente {c}"
                ).add_to(mapa)

    # Crear leyenda
    legend_html = """
    <div style='position: fixed; bottom: 20px; left: 20px; z-index:9999; background-color:white;
                padding: 15px; border:2px solid grey; border-radius:8px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);'>
      <h4 style='margin-top: 0; margin-bottom: 10px; color: #333;'>🚛 Vehículos Activos</h4>
    """

    vehiculos_activos = 0
    for v in model.V:
        if rutas_por_vehiculo[v]:  # Solo mostrar vehículos con clientes
            vehiculos_activos += 1
            num_clientes = len(rutas_por_vehiculo[v])
            carga = sum(clientes[c]['demanda'] for c in rutas_por_vehiculo[v])
            capacidad = vehiculos[v]['capacidad']

            legend_html += f"""
            <div style='margin-bottom: 5px;'>
                <i style='background:{color_vehiculo[v]};width:12px;height:12px;display:inline-block;border-radius:2px;'></i>
                <strong>{v}</strong>: {num_clientes} clientes ({carga}/{capacidad})
            </div>
            """

    legend_html += f"<hr><small><strong>Total:</strong> {vehiculos_activos} vehículos utilizados</small></div>"
    mapa.get_root().html.add_child(folium.Element(legend_html))

    # Agregar información del problema en la esquina superior derecha
    info_html = f"""
    <div style='position: fixed; top: 20px; right: 20px; z-index:9999; background-color:white;
                padding: 10px; border:2px solid grey; border-radius:6px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);'>
      <h5 style='margin-top: 0; color: #333;'>📊 Resumen</h5>
      <small>
        <strong>Clientes:</strong> {len(model.C)}<br>
        <strong>Vehículos disponibles:</strong> {len(model.V)}<br>
        <strong>Vehículos utilizados:</strong> {vehiculos_activos}<br>
        <strong>Costo total:</strong> ${pyo.value(model.obj):,.0f}
      </small>
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(info_html))

    return mapa

def mostrar_estadisticas_solucion(model, centros, clientes, vehiculos):
    """Muestra estadísticas detalladas de la solución."""
    print("\n" + "="*50)
    print("📈 ESTADÍSTICAS DE LA SOLUCIÓN")
    print("="*50)

    vehiculos_usados = 0
    carga_total_usada = 0
    capacidad_total_disponible = sum(vehiculos[v]['capacidad'] for v in vehiculos)

    for v in model.V:
        clientes_asignados = []
        carga_vehiculo = 0

        for c in model.C:
            if pyo.value(model.x[c, v]) == 1:
                clientes_asignados.append(c)
                carga_vehiculo += clientes[c]['demanda']

        if clientes_asignados:
            vehiculos_usados += 1
            carga_total_usada += carga_vehiculo
            utilizacion = (carga_vehiculo / vehiculos[v]['capacidad']) * 100

            print(f"\n🚛 {v}:")
            print(f"   Clientes: {', '.join(clientes_asignados)}")
            print(f"   Carga: {carga_vehiculo}/{vehiculos[v]['capacidad']} ({utilizacion:.1f}%)")

    eficiencia_flota = (carga_total_usada / capacidad_total_disponible) * 100

    print(f"\n📊 RESUMEN GENERAL:")
    print(f"   Vehículos utilizados: {vehiculos_usados}/{len(vehiculos)}")
    print(f"   Eficiencia de la flota: {eficiencia_flota:.1f}%")
    print(f"   Costo total: ${pyo.value(model.obj):,.2f}")
    print("="*50)


In [ ]:
centros = leer_datos_centro_caso1('depotsC3.csv')
clientes = leer_datos_clientes('clientsC3.csv')
vehiculos = leer_datos_vehiculos('vehiclesC3.csv')


print(centros)
print(clientes)
print(vehiculos)


# Modelo simplificado (rápido pero mejor que heurística)
model = modelo_super_rapido(centros, clientes, vehiculos)

mapa = mostrar_mapa_solucion_simple(model, centros, clientes, vehiculos)
mapa.save("mapa_rutas.html")
mapa

{'CD1': {'lat': 4.75021190869025, 'lon': -74.08124218159384, 'capacidad': 11.0}, 'CD2': {'lat': 4.5363832206427785, 'lon': -74.10993358606953, 'capacidad': 90.0}, 'CD3': {'lat': 4.792925960208614, 'lon': -74.03854814565923, 'capacidad': 130.0}, 'CD4': {'lat': 4.72167778077445, 'lon': -74.06706883098641, 'capacidad': 145.0}, 'CD5': {'lat': 4.607707046760958, 'lon': -74.13826337931849, 'capacidad': 260.0}, 'CD6': {'lat': 4.650463053612691, 'lon': -74.12400186370824, 'capacidad': 180.0}, 'CD7': {'lat': 4.621911772492814, 'lon': -74.09561875464892, 'capacidad': 720.0}, 'CD8': {'lat': 4.678960680833056, 'lon': -74.10975623736951, 'capacidad': 55.0}, 'CD9': {'lat': 4.735973062153282, 'lon': -74.09547235719887, 'capacidad': 70.0}, 'CD10': {'lat': 4.550640992537941, 'lon': -74.10991610076434, 'capacidad': 75.0}, 'CD11': {'lat': 4.664702960902753, 'lon': -74.10977422186991, 'capacidad': 90.0}, 'CD12': {'lat': 4.5791740634103135, 'lon': -74.12408925943333, 'capacidad': 270.0}}
{'C1': {'lat': 4.6